# Image Classification Training Pipeline

In this sample, image preprocessing is on cpu nodes while training on distributed gpu nodes.

## Configure workspace and compute

In [ ]:
# configure workspace information here.
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
# forced_interactive_auth = InteractiveLoginAuthentication(tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47", force=True)

ws = Workspace.get(
    name='gjd-test',
    subscription_id='f92fab0a-38b9-44ed-b800-2ed55fa1b9d9',
    resource_group='Groot-Shared'
    # auth=forced_interactive_auth
)

## Prepare dataset
This smaller imagenet dataset is a subset of the official one.
- training dataset contains 1.2m images (1000 categories * 1200 images per category)
- validation dataset contains 50k images (1000 categories * 50 images per category)

Need to use zip file here to avoid perf issue of mounting file dataset with many subfolders.

In [ ]:
# get dataset
from azureml.core.dataset import Dataset

train_image_dataset = Dataset.get_by_name(ws, name='imagenet_train_dataset')
val_image_dataset = Dataset.get_by_name(ws, name='imagenet_valid_dataset')

## Load modules

In [ ]:
from azure.ml.component import Component

# load built-in components
convert_func, init_transform_func, apply_transform_func = Component.batch_load(ws, selectors=['azureml://Convert to Image Directory', 'azureml://Init Image Transformation', 'azureml://Apply Image Transformation'])

In [ ]:
train_module = Component.from_yaml(ws, yaml_file='modules/ConvNets/entry.spec.yaml')

## Set up a pipeline

In [ ]:
# define pipeline
# define pipeline
from azure.ml.component import dsl

compute_target = "eus-p40-5-gjd"
datastore_name = 'chjinche_adls_gen1'

@dsl.pipeline(name='image classification', description='image classification', default_compute_target=compute_target,
default_datastore=datastore_name)
def generated_pipeline():
    convert_train = convert_func(
        input_dataset=train_image_dataset
    )
    
    convert_val = convert_func(
        input_dataset=val_image_dataset
    )
    
    init_trans = init_transform_func(
        resize='False',
        size=256,
        center_crop='False',
        crop_size=224,
        pad='False',
        padding=0,
        color_jitter='False',
        grayscale='False',
        random_resized_crop='False',
        random_resized_crop_size=256,
        random_crop='False',
        random_crop_size=224,
        random_horizontal_flip='True',
        random_vertical_flip='False',
        random_rotation='False',
        random_rotation_degrees=0,
        random_affine='False',
        random_affine_degrees=0,
        random_grayscale='False',
        random_perspective='False'
    )
    
    apply_trans_on_train = apply_transform_func(
        mode='For training',
        input_image_transformation=init_trans.outputs.output_image_transformation,
        input_image_directory=convert_train.outputs.output_image_directory
    )
    
    apply_trans_on_val = apply_transform_func(
        mode='For inference',
        input_image_transformation=init_trans.outputs.output_image_transformation,
        input_image_directory=convert_val.outputs.output_image_directory
    )
    
    train = train_module(
        train_data=apply_trans_on_train.outputs.output_image_directory,
        valid_data=apply_trans_on_val.outputs.output_image_directory,
        data_backend='pytorch',
        pretrained_weights=None,
        epochs=2,
        seed=123,
        batch_size=16,
        print_freq=100,
        # lr=4.096,
        # optimizer_batch_size=4096,
        save_checkpoint_epochs=1
    )
    # perform distributed training with 4 nodes.
    # note: process_count_per_node should be 1 because this module will launch distributed processes based on node device count.
    train.runsettings.configure(target=compute_target, node_count=2, process_count_per_node=4)

In [ ]:
# create a pipeline
pipeline = generated_pipeline()

In [ ]:
# validate pipeline and visualize the graph
pipeline.validate()

In [ ]:
pipeline.submit(experiment_name='test-gen1-itp-image-classification')